# 🖼️ Text Classification Playground  
Text classification with distilbert trained on `rotten_tomatoes` dataset.

1. **Run the first cell** to install requirements.  
2. Switch the runtime to **GPU**. If running on Colab Runtime → Change runtime type → T4 GPU.
3. Enter a prompt to test the trained model.

> Model: *[Custom Distilbert Rotten Tomatoes](https://huggingface.co/dbilgin/distilbert-rotten-tomatoes)*.

In [ ]:
%pip install -q -r https://raw.githubusercontent.com/dbilgin/ai_playground/refs/heads/master/requirements.txt

In [ ]:
import gradio as gr
from transformers.pipelines import pipeline

gen = pipeline(
    "text-classification",
    model="dbilgin/distilbert-rotten-tomatoes",
    model_kwargs={
        "load_in_4bit": True,
        "device_map": {"": 0},
        "max_memory": {0: "10GiB", "cpu": "32GiB"},
        "torch_dtype": "auto"
    }
)

def predict_sentiment(text):
    result = gen(text)[0]
    return f"Label: {result['label']}, Score: {result['score']:.4f}"

gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(label="Enter movie review", placeholder="This movie was amazing!"),
    outputs=gr.Textbox(label="Sentiment Analysis Result"),
    title="Rotten Tomatoes Sentiment Analysis"
).launch()

# Train

- Run the below cell to train the [distilbert-base-uncased](https://huggingface.co/distilbert/distilbert-base-uncased) model on rotten_tomatoes data.
- Uncomment `trainer.push_to_hub()` if you would like to upload the result to Hugging Face.

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset, DatasetDict
from transformers.data.data_collator import DataCollatorWithPadding
from transformers.training_args import TrainingArguments
from transformers.trainer import Trainer

model_name = "distilbert/distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
dataset = load_dataset("rotten_tomatoes")

def tokenize_dataset(dataset):
    return tokenizer(dataset["text"])
dataset = dataset.map(tokenize_dataset, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="trained_models/distilbert-rotten-tomatoes",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    push_to_hub=False, # Enable to push to hub
)

if not isinstance(dataset, DatasetDict):
    raise ValueError("Dataset must be a DatasetDict")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

# Enable to push to hub
# trainer.push_to_hub()
